In [ ]:
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

opts = Options()
#opts.add_argument('--headless')  #無頭chrome
opts.add_argument('--disable-gpu')

driver = webdriver.Chrome(executable_path=ChromeDriverManager().install(),chrome_options=opts)

driver.get('http://www56.eyny.com/member.php?mod=logging&action=login')
element = driver.find_element_by_xpath("(//input[@class='px p_fre'])[1]")
#帳號
element.send_keys("帳號")
element = driver.find_element_by_xpath("(//input[@class='px p_fre'])[2]")
#密碼
element.send_keys("密碼")
element = driver.find_element_by_xpath("(//button[@class='pn pnc'])[2]")
element.click()
time.sleep(5)
driver.get('http://www56.eyny.com/forum.php?mod=forumdisplay&fid=2')
element = driver.find_element_by_xpath("//input[@type='submit']")
element.click()

#頁面
title=[]
title_url_o=[]
up_time=[]
response_num=[]
views_sum=[]
for i in range(50):
    try:
        titles = driver.find_element_by_xpath(f"(//tbody[@id]//th[@class='new']/a)[{i}]")
        times = driver.find_element_by_xpath(f"(//tbody[@id]//td[@class='by']/em/span)[{i}]")
        response_nums = driver.find_element_by_xpath(f"(//tbody[@id]//td[@class='num']/a)[{i}]")
        views_sums = driver.find_element_by_xpath(f"(//tbody[@id]//td[@class='num']/em)[{i}]")

        title.append(titles.text)
        title_url_o.append(titles.get_attribute('href'))
        up_time.append(times.text)
        response_num.append(response_nums.text)
        views_sum.append(views_sums.text)


    except:
        pass
        #print(f'------ {i} end ------')

for p in range(1,3):
    element = driver.find_element_by_xpath("//a[@class='nxt']")
    element.click()

    for i in range(50):
        try:
            titles = driver.find_element_by_xpath(f"(//tbody[@id]//th[@class='common']/a)[{i}]")
            times = driver.find_element_by_xpath(f"(//tbody[@id]//td[@class='by']/em/span)[{i}]")
            response_nums = driver.find_element_by_xpath(f"(//tbody[@id]//td[@class='num']/a)[{i}]")
            views_sums = driver.find_element_by_xpath(f"(//tbody[@id]//td[@class='num']/em)[{i}]")

            title.append(titles.text)
            title_url_o.append(titles.get_attribute('href'))
            up_time.append(times.text)
            response_num.append(response_nums.text)
            views_sum.append(views_sums.text)
        except:
            pass

#title_url = [i for i in title_url_o if len(i)<65]
#title_thunmb = [i for i in title_url_o if len(i)>65]

video_type=[]
video_ID=[]
download_type_o=[]
pic_number=[]
for i in range(len(title)):
    video_type.append(re.findall('\[\w+\]',title[i]))
    video_ID.append(re.findall('\w+-\w+',title[i]))
    download_type_o.append(re.findall('\@\w+',title[i]))
    pic_number.append(re.findall('\(\w+\)',title[i]))


#載點
download_type_u=[]
for i in range(len(download_type_o)):
    try:
        download_type_u.append(download_type_o[i][0])
    except:
        download_type_u.append('')
download_type=[]
for i in range(len(download_type_u)):
    try:
        download_type.append(re.sub('\@','',download_type_u[i]))
    except:
        download_type.append('')

#馬
horse_o=[]
for i in range(len(download_type_o)):
    try:
        horse_o.append(download_type_o[i][1])
    except:
        horse_o.append('')
horse=[]
for i in range(len(horse_o)):
    try:
        horse.append(re.sub('\@','',horse_o[i]))
    except:
        horse.append('')


#內文test
page_text=[]
page_text2=[]
page_url=[]
page_url2=[]
for i in title_url_o:
    try:
        #文章內容分兩種格式
        try:
            #文章格式1
            driver.get(i)
            time.sleep(0.1)
            page_texts = driver.find_element_by_xpath(f"(//td[@class='t_f'])[1]")
            page_text.append(page_texts.text)
            time.sleep(0.1)
        except:
            #文章格式2
            driver.get(i)
            time.sleep(0.1)
            for i in range(100):
                try:
                    page_texts = driver.find_element_by_xpath(f"((//td[@class='t_f'])[1]/p)[{i}]")
                    page_text2.append(page_texts.text)
                    time.sleep(0.1)
                except:
                    pass

        #url
        
        for i in range(50):
            try:
                page_urls = driver.find_element_by_xpath(f"((//td[@class='t_f'])[1]/a)[{i}]")
                page_url2.append(page_urls.get_attribute('href'))
            except:
                pass

    except:
        page_text.append('')
        page_url.append('')




#表格
df = pd.DataFrame({'標題':title,
                   '影片類型':video_type,
                   '影片ID':video_ID,
                   '下載資訊':download_type,
                   '預覽圖數量':pic_number,
                   '上傳時間':up_time,
                   '回覆次數':response_num,
                   '查看次數':views_sum,
                   '文章連結':title_url_o})

df = df.drop(df[df['標題']=='New'].index)

df

#driver.close()


In [ ]:
df.to_csv("/Users/syunhua/Desktop/000_prase_python/eyny_crawler.csv")